In [1]:
import numpy as np
import pandas as pd
import re
import string
import nltk
from nltk.util import ngrams
from nltk.tokenize import RegexpTokenizer
from sklearn.feature_extraction.text import TfidfVectorizer
import spacy 
from spacy import displacy
from collections import Counter

In [2]:
#test strings
en_string = "This list has overlapping features with content features. For example, word n-grams will capture the content of the text along with stylometric tendencies. Content features consist of word frequencies, word and character n-grams, hapax legomena etc. This overlap is not of concern, however, as Sari et al. show, using content features is beneficial when performing authorship attribution of news articles because journalists often have certain topics they prefer writing about. They argue that using only stylometric features is beneficial when attributing authors to texts of the same topic or genre, e.g. law text or movie reviews."
da_test = 'Cecilia Lonning-Skovgaard har tidligere oplyst, at hun har søgt professionel hjælp til at forbedre sin ledelsesstil.En undersøgelse af det psykiske arbejdsmiljø i centralforvaltningen i Københavns kommune er "rystende læsning" og vidner om store svigt i toppen af ledelsen - herunder særligt beskæftigelses- og Integrationsborgmester Cecilia Lonning-Skovgaard (V). Sådan lyder det i et åbent brev fra en række fagforbund. Undersøgelsen dokumenterer en omfattende og fuldkommen uacceptabel, krænkende adfærd fra borgmesterens side og fjerner den sidste rest af tvivl om, hvor alvorlig og hvor uholdbar situationen er. Af undersøgelsen foretaget blandt 188 medarbejdere fremgår det, at 27 procent har oplevet krænkende adfærd, og 30 procent har været vidne til krænkende adfærd i centralforvaltningen. 47 procent af dem, der har været udsat for krænkelser og 68 procent af dem, der har været vidne til krænkende adfærd, svarer, at det er ”borgmesteren eller den øvrige politiske ledelse”, der står bag den krænkende adfærd.'

da_string = ['Cecilia Lonning-Skovgaard har tidligere oplyst, at hun har søgt professionel hjælp til at forbedre sin ledelsesstil.En undersøgelse af det psykiske arbejdsmiljø i centralforvaltningen i Københavns kommune er "rystende læsning" og vidner om store svigt i toppen af ledelsen - herunder særligt beskæftigelses- og Integrationsborgmester Cecilia Lonning-Skovgaard (V). Sådan lyder det i et åbent brev fra en række fagforbund. Undersøgelsen dokumenterer en omfattende og fuldkommen uacceptabel, krænkende adfærd fra borgmesterens side og fjerner den sidste rest af tvivl om, hvor alvorlig og hvor uholdbar situationen er. Af undersøgelsen foretaget blandt 188 medarbejdere fremgår det, at 27 procent har oplevet krænkende adfærd, og 30 procent har været vidne til krænkende adfærd i centralforvaltningen. 47 procent af dem, der har været udsat for krænkelser og 68 procent af dem, der har været vidne til krænkende adfærd, svarer, at det er ”borgmesteren eller den øvrige politiske ledelse”, der står bag den krænkende adfærd.']

train_corpus =['Til TV 2 oplyser Camilla Gregersen, formand for den akademiske fagforening DM, at Venstre bør overveje, om Cecilia Lonning-Skovgaard er den rette til posten.',
              '- Der er et kæmpe problem i forvaltningen, og det er centreret omkring den øverste ledelse. Det sender dårlig energi i hele systemet, og det er man nødt til at handle på nu.',
              'TV 2 har forsøgt at få en kommentar fra Venstres formand Jakob Ellemann-Jensen for at høre, hvordan han forholder sig til undersøgelsen og Cecilia Lonning-Skovgaards fremtid.']

test_corpus = ['I brevet fra fagforbundene lyder opfordringen desuden, at Beskæftigelses- og Integrationsudvalget i Københavns Kommune "øjeblikkeligt" skal få styr på det dårlige arbejdsmiljø under borgmesteren.',
               'TV 2 forsøger at få en kommentar fra Cecilia Lonning-Skovgaard.', 
               'Til Politiken siger hun:- Det tager jeg et kæmpe ansvar for, og det har jeg også sagt til medarbejderne her til morgen og undskyldt for. Jeg er i fuld gang med at arbejde på tonen over for medarbejderne, og jeg er også stoppet med at tage direkte kontakt til medarbejderne.',
               'Borgmesteren har tidligere oplyst, at hun har søgt professionel hjælp til at forbedre sin ledelsesstil.',
               'Undersøgelsen af det psykiske arbejdsmiljø blev igangsat i januar, efter at HK og DJØF i et åbent brev kritiserede Cecilia Lonning-Skovgaard for manglende indsigt i sin forvaltning og manglende forståelse for sin rolle som borgmester.']

In [3]:
def remove_punctuation(text):
    return re.sub(r',|\.|:|!|\?|;', '', text)

In [4]:
# def create_ngrams(text, n): 
#     no_punct_text = remove_punctuation(text)
    
#     #create ngrams of words
#     word_ngrams = ngrams(no_punct_text.split(), n)
#     ngram_list = []
#     for ngram in word_ngrams: 
#         ngram_list.append(ngram)
    
#     #create character ngrams
#     no_space_text = no_punct_text.replace(" ","")
#     char_list = list(no_space_text.lower())
#     char_ngrams = ngrams(char_list, n)
    
#     char_ngram_list = []
#     for char_ngram in char_ngrams: 
#         char_ngram_list.append(char_ngram)
    
#     output = {"ngrams": ngram_list, 
#               "char_ngrams": char_ngram_list}
    
#     return output

In [5]:
#POS tagger trained on Danish news and media corpus
POS_tagger_DK = spacy.load("da_core_news_md")

In [6]:
#Trains a TF-IDF vectorizer of word n-grams
def word_ngram_vectorizer(train_corpus, n): 
    
    vectorizer = TfidfVectorizer(analyzer="word", ngram_range=(n,n))
    X = vectorizer.fit_transform(train_corpus)
    ngrams = vectorizer.get_feature_names()
    dense = X.todense()
    denselist = dense.tolist()
    df = pd.DataFrame(denselist, columns=ngrams)
    display(df.head(2))
      
    return X, vectorizer

#Trains a TF-IDF vectorizer of character n-grams
def char_ngram_vectorizer(train_corpus, n): 
    
    vectorizer = TfidfVectorizer(analyzer="char", ngram_range=(n,n))
    X = vectorizer.fit_transform(train_corpus)
    ngrams = vectorizer.get_feature_names()
    dense = X.todense()
    denselist = dense.tolist()
    df = pd.DataFrame(denselist, columns=ngrams)
    display(df.head(2))
      
    return X, vectorizer

#Trains a TF-IDF vectorizer of POS n-grams. A POS corpus is generated in the function using a tagger for Danish
def POS_ngram_vectorizer(train_corpus, n): 
    
    #Create POS corpus
    POS_corpus = []

    for doc in train_corpus:
        tagged_doc = POS_tagger_DK(doc) #tag each document in corpus with POS tags using spacy
        POS_list = []

        for token in tagged_doc:
            POS_list.append(token.pos_)

        #concatenate as POS tags for the document
        POS_text = " ".join(POS_list)
        POS_corpus.append(POS_text)

    
    vectorizer = TfidfVectorizer(analyzer="word", ngram_range=(n,n))
    X = vectorizer.fit_transform(POS_corpus)
    ngrams = vectorizer.get_feature_names()
    dense = X.todense()
    denselist = dense.tolist()
    df = pd.DataFrame(denselist, columns=ngrams)
    display(df.head(2))
    
    # Returns 
    return X, vectorizer

# Gets weights for terms based on trained vectorizer
# Works for both word and character ngrams
def get_tfidf_ngrams(vectorizer, test_corpus):
    '''Returns the TF-IDF weighted ngram frequencies of test documents'''
    #Multiple texts required
    return vectorizer.transform(test_corpus)

# Function generates POS test corpus first and then gts weights for terms based on trained vectorizer. 
def get_tfidf_POS_ngrams(vectorizer, test_corpus):
    '''Returns the TF-IDF weighted ngram frequencies of test documents'''
    #Create POS corpus
    POS_corpus = []

    for doc in test_corpus:
        tagged_doc = POS_tagger_DK(doc) #tag each document in corpus with POS tags using spacy
        POS_list = []

        for token in tagged_doc:
            POS_list.append(token.pos_)

        #concatenate as POS tags for the document
        POS_text = " ".join(POS_list)
        POS_corpus.append(POS_text)
    
    #Multiple texts required
    return vectorizer.transform(POS_corpus)
    


In [7]:
X, POS_trigram_vectorizer = POS_ngram_vectorizer(train_corpus,3)

,adj adp part,adj noun adp,adj noun propn,adj noun punct,adp adj noun,adp adv punct,adp det adj,adp noun cconj,adp noun noun,adp noun num,...,verb adj noun,verb adp adv,verb adp det,verb det adj,verb det noun,verb part verb,verb pron adp,verb propn propn,verb punct adv,verb punct sconj
0,0.000000,0.000000,0.205575,0.000000,0.000000,0.000000,0.156345,0.0,0.0,0.205575,...,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.205575,0.0,0.205575
1,0.148417,0.296834,0.000000,0.296834,0.148417,0.148417,0.112875,0.0,0.0,0.000000,...,0.148417,0.148417,0.148417,0.148417,0.0,0.0,0.0,0.000000,0.0,0.000000


In [10]:
POS_trigram_vector = get_tfidf_POS_ngrams(POS_trigram_vectorizer, test_corpus).toarray()
POS_trigram_vector

array([[0.        , 0.40387946, 0.        , 0.        , 0.        ,
        0.        , 0.30716074, 0.        , 0.        , 0.        ,
        0.30716074, 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.30716074, 0.        , 0.        , 0.40387946, 0.47707544,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.40387946, 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        ],
   